# Download Company's Updated Annual Report XBRL

In this project, I will try to automate the downloading of company's updated annual report from [SEC website](https://www.sec.gov/edgar/searchedgar/companysearch.html). The annual report will be downloaded as XBRL instance document.

# Create Functions

First, I will create the functions that I needed.

In [1]:
import requests
import time
from bs4 import BeautifulSoup
import datetime as dt
import dateutil.relativedelta as du

def search_company_from_sec_edgar(cik):
    is_correct_page_title = False
    is_correct_cik = False
    is_col_filings_found = False
    is_col_format_found = False
    is_col_description_found = False
    is_col_filing_date_found = False
    is_correct_page = False

    # search company from SEC EDGAR system using central index key (CIK)
    i_cik = int(cik)
    url = 'https://www.sec.gov/cgi-bin/browse-edgar?CIK={}'.format(i_cik)

    print('Go to {}'.format(url))
    response = requests.get(url)
    content = response.content

    # verify that arrived at EDGAR Search Results page
    parser = BeautifulSoup(content, 'html.parser')

    # verify page title
    page_title = parser.select('#PageTitle')[0]

    if page_title.text.upper() == 'EDGAR Search Results'.upper():
        is_correct_page_title = True

    # verify central index key
    span_companyName = parser.select('span.companyName')[0]

    if cik in span_companyName.text:
        is_correct_cik = True

    # verify table column headers
    table = parser.select('#seriesDiv > table')[0]

    col_number = 0

    for th in table.select('th'):
        if th.text.upper() == 'Filings'.upper():
            is_col_filings_found = True
        elif th.text.upper() == 'Format'.upper():
            is_col_format_found = True
        elif th.text.upper() == 'Description'.upper():
            is_col_description_found = True
        elif th.text.upper() == 'Filing Date'.upper():
            is_col_filing_date_found = True

        col_number += 1

    if is_correct_page_title \
    and is_correct_cik \
    and is_col_filings_found \
    and is_col_format_found \
    and is_col_description_found \
    and is_col_filing_date_found:
        is_correct_page = True

    if not is_correct_page:
        raise Exception('Not at EDGAR Search Results page.')
    
    # return the table from EDGAR Search Results page
    return table
    
    
def go_to_filing_detail_page(filing_detail_url):
    is_correct_page_title = False
    is_col_description_found = False
    is_col_document_found = False
    is_correct_page = False
    
    # wait for 10 seconds before request again to avoid being blocked
    # due to too many requests within a short time
    print('Wait for 10 seconds ...')
    time.sleep(10)

    print('Go to {}'.format(filing_detail_url))
    response = requests.get(filing_detail_url)
    content = response.content
    
    parser = BeautifulSoup(content, 'html.parser')
    
    # verify page title
    page_title = parser.select('#PageTitle')[0]
    
    if page_title.text.upper() == 'Filing Detail'.upper():
        is_correct_page_title = True
    
    # verify table column headers
    table = parser.select("table[summary='Data Files']")[0]
    
    for th in table.select('tr > th'):
        if th.text.upper() == 'Description'.upper():
            is_col_description_found = True
        elif th.text.upper() == 'Document'.upper():
            is_col_document_found = True

    if is_correct_page_title and is_col_description_found and is_col_document_found:
        is_correct_page = True
        
    if not is_correct_page:
        raise Exception('Not at Filing Detail page.')
    
    # get URL of XBRL instance document
    file_partial_url = ''
    file_name = ''

    for tr in table.select('tr'):
        tds = tr.select('td')

        if len(tds) > 0 and ('XBRL INSTANCE DOCUMENT' in tds[1].text):
            a = tds[2].select('a')[0]
            file_name = a.text
            file_partial_url = a['href']
            
    if len(file_partial_url) == 0:
        raise Exception('URL of XBRL instance document not found.')
    
    # return URL and file name of XBRL instance document from Filing Detail page
    return file_partial_url, file_name


def check_for_updated_annual_report(cik, fy_ended):
    # search company from SEC EDGAR system using central index key (CIK)
    table = search_company_from_sec_edgar(cik)

    # from EDGAR Search Results page, check whether there is updated annual report
    twelve_mths_after_fy_ended = fy_ended + du.relativedelta(months=12)
    is_updated_annual_report_available = False
    filing_detail_partial_url = None

    for tr in table.select('tr'):
        tds = tr.select('td')

        if len(tds) > 0:
            description = tds[2].text

            # search for the row with annual report description and 'Interactive Data' button
            if description.startswith('Annual') and 'report' in description \
            and len(tds[1].select('a#interactiveDataBtn')):
                filing_date = tds[3].text
                dt_filing_date = dt.datetime.strptime(filing_date, '%Y-%m-%d')
                documents_button = tds[1].select('a#documentsbutton')[0]

                # check whether filing date is 12 months after the noted fiscal year ended
                if dt_filing_date > twelve_mths_after_fy_ended:
                    is_updated_annual_report_available = True
                    filing_detail_partial_url = documents_button['href']

    return is_updated_annual_report_available, filing_detail_partial_url


def get_updated_annual_report_XBRL(cik, fy_ended, download_to_directory):
    file_path = None
    
    is_updated_annual_report_available \
    , filing_detail_partial_url = check_for_updated_annual_report(cik, fy_ended)

    # if there is updated annual report, download XBRL instance document
    if is_updated_annual_report_available:
        domain_url = 'https://www.sec.gov'
        filing_detail_url = domain_url + filing_detail_partial_url

        file_partial_url, file_name = go_to_filing_detail_page(filing_detail_url)

        # download XBRL instance document
        file_url = domain_url + file_partial_url
        file_path = download_to_directory + '/' + file_name

        # wait for 10 seconds before request again to avoid being blocked 
        # due to too many requests within a short time
        print('Wait for 10 seconds ...')
        time.sleep(10)

        print('Download {} from {} to {}'.format(file_name, file_url, file_path))
        response = requests.get(file_url)

        with open(file_path, 'wb') as f:
            f.write(response.content)
            
    return is_updated_annual_report_available, file_path

# Test Function

In [2]:
import sqlalchemy as sa
import pandas as pd
import configparser
import os

# get configuration from config file
config = configparser.ConfigParser()
config.read('config.ini')
#config.read(os.path.dirname(__file__) + '/config.ini') # use this line when running from cron job
uid = config['Sql']['uid']
pwd = config['Sql']['pwd']
host = config['Sql']['host']
port = config['Sql']['port']
download_to_directory = config['Path']['download_to_directory']

# retrieve rows with fiscal year ended more than 12 months ago
engine = sa.create_engine('postgresql://{}:{}@{}:{}/stock'.format(uid, pwd
                                                                  , host, port))
sql = "SELECT \
            company_name, fiscal_year_ended, central_index_key \
        from company \
        where \
            fiscal_year_ended < (current_date - interval '12 months') \
"
df = pd.read_sql(sql, con=engine, parse_dates=['fiscal_year_ended'])
print(df, '\n')

# process rows
if len(df) > 0:
    for index, row in df.iterrows():
        cik = row['central_index_key']
        fy_ended = row['fiscal_year_ended']
        
        print('Processing for CIK {}, fiscal year ended {}.' \
              .format(cik, fy_ended.strftime('%d %b %Y')))
        
        is_updated_annual_report_available \
        , file_path = get_updated_annual_report_XBRL(cik, fy_ended, download_to_directory)
        
        print('Is updated annual report available:', is_updated_annual_report_available)

         company_name fiscal_year_ended central_index_key
0  ATLASSIAN CORP PLC        2019-06-30        0001650372 

Processing for CIK 0001650372, fiscal year ended 30 Jun 2019.
Go to https://www.sec.gov/cgi-bin/browse-edgar?CIK=1650372
Is updated annual report available: False


Test using the Central Index Key (CIK) of ABIOMED, Inc. (ABMD).

In [3]:
import datetime as dt

cik = '0000815094'
fy_ended = dt.datetime(2019, 3, 31)

is_updated_annual_report_available \
, file_path = get_updated_annual_report_XBRL(cik, fy_ended, download_to_directory)

Go to https://www.sec.gov/cgi-bin/browse-edgar?CIK=815094
Wait for 10 seconds ...
Go to https://www.sec.gov/Archives/edgar/data/815094/000156459020026547/0001564590-20-026547-index.htm
Wait for 10 seconds ...
Download abmd-10k_20200331_htm.xml from https://www.sec.gov/Archives/edgar/data/815094/000156459020026547/abmd-10k_20200331_htm.xml to /Users/Steffen/Extract_financial_data_from_XBRL/data_files/abmd-10k_20200331_htm.xml
